In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('Dataset.csv')
df.head()

,Unnamed: 0,over,ball,total_runs,wicket left,runs required,winner,season
0,0,1,1,1,10.0,206.0,0.0,2017
1,1,1,2,1,10.0,206.0,0.0,2017
2,2,1,3,1,10.0,206.0,0.0,2017
3,3,1,4,3,10.0,204.0,0.0,2017
4,4,1,5,7,10.0,200.0,0.0,2017


In [3]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [4]:
df.head()

,over,ball,total_runs,wicket left,runs required,winner,season
0,1,1,1,10.0,206.0,0.0,2017
1,1,2,1,10.0,206.0,0.0,2017
2,1,3,1,10.0,206.0,0.0,2017
3,1,4,3,10.0,204.0,0.0,2017
4,1,5,7,10.0,200.0,0.0,2017


In [5]:
def create_dataset(dataset,time_stamp=6):
    X,Y = [],[]
    for i in range(len(dataset)-time_stamp-1):
        a = dataset[i:(i+time_stamp),:-1]
        X.append(a)
        Y.append(dataset[i+time_stamp,-1])
    return np.array(X), np.array(Y)

In [6]:
train_df = df[df['season']<=2017]
test_df = df[df['season']>2017]

In [7]:
train_df.drop(['season'],axis=1,inplace=True)
test_df.drop(['season'],axis=1,inplace=True)

In [8]:
stamp=6
x_train, y_train = create_dataset(train_df.values, time_stamp=stamp)

x_test, y_test = create_dataset(test_df.values, time_stamp=stamp)

In [9]:
x_train.shape

(70482, 6, 5)

In [10]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(6,5)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10,validation_data=(x_test, y_test))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 70482 samples, validate on 13437 samples
Epoch 1/10
70482/70482 [==============================] - 50s 710us/sample - loss: 0.4888 - acc: 0.7524 - val_loss: 0.5382 - val_acc: 0.7060
Epoch 2/10
70482/70482 [==============================] - 43s 606us/sample - loss: 0.4596 - acc: 0.7741 - val_loss: 0.5484 - val_acc: 0.6951
Epoch 3/10
70482/70482 [==============================] - 43s 608us/sample - loss: 0.4540 - acc: 0.7764 - val_loss: 0.5960 - val_acc: 0.6967
Epoch 4/10
70482/70482 [==============================] - 42s 599us/sample - loss: 0.4527 - acc: 0.7769 - val_loss: 0.5591 - val_acc: 0.6926
Epoch 5/10
70482/70482 [==============================] - 44s 619us/sample - loss: 0.4518 - acc: 0.7767 - val_loss: 0.5448 - val_acc: 0.6935
Epoch 6/10
70482/70482 [==============================] - 43s 612us/sa

In [11]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

predictions = probability_model.predict(x_test)

In [12]:
model.save('ipl.h5')